In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pytorch_lightning as pl
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.classification import MulticlassF1Score, MulticlassAUROC

In [2]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Random seed set as {seed}")


set_seed(42)

Random seed set as 42


# Создание класса `FashionMNISTDataModule`

In [3]:
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir="./data", batch_size=64):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

        self.transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
        )

    def prepare_data(self):
        torchvision.datasets.FashionMNIST(self.data_dir, train=True, download=True)
        torchvision.datasets.FashionMNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            fashion_full = torchvision.datasets.FashionMNIST(
                self.data_dir, train=True, transform=self.transform
            )
            self.fashion_train, self.fashion_val = random_split(
                fashion_full, [55000, 5000], generator=torch.Generator().manual_seed(42)
            )

        if stage == "test" or stage is None:
            self.fashion_test = torchvision.datasets.FashionMNIST(
                self.data_dir, train=False, transform=self.transform
            )

    def train_dataloader(self):
        return DataLoader(self.fashion_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.fashion_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.fashion_test, batch_size=self.batch_size)

# Создание класса модели `FashionMNIST`

In [4]:
class FashionMNIST(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters()

        self.f1 = MulticlassF1Score(num_classes=10)
        self.auroc = MulticlassAUROC(num_classes=10)

        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_f1", self.f1(preds, y), prog_bar=True)
        self.log("val_auroc", self.auroc(logits, y), prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)

        self.log("test_loss", loss, prog_bar=True)
        self.log("test_f1", self.f1(preds, y), prog_bar=True)
        self.log("test_auroc", self.auroc(logits, y), prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode="min", factor=0.1, patience=3
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"},
        }

# Обучение модели

In [5]:
logger = TensorBoardLogger("lightning_logs", name="fashion_mnist")
early_stopping = EarlyStopping(monitor="val_loss", patience=5, mode="min")

data_module = FashionMNISTDataModule()
model = FashionMNIST()

trainer = pl.Trainer(
    max_epochs=5,  # Обучаю на CPU =)
    callbacks=[early_stopping],
    logger=logger,
    accelerator="auto",
    devices=1,
)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model, data_module)

100%|██████████| 26.4M/26.4M [00:01<00:00, 20.0MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 1.08MB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 16.0MB/s]


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [7]:
trainer.test(model, data_module)

/home/nikolai/.cache/pypoetry/virtualenvs/itmo-dl-EUHjMda--py3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.9938201308250427
         test_f1            0.9096488356590271
        test_loss           0.22786672413349152
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.22786672413349152,
  'test_f1': 0.9096488356590271,
  'test_auroc': 0.9938201308250427}]

# Логи

In [9]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

Launching TensorBoard...